In [1]:
import numpy as np

def price_bermudan_call_binomial(S0, K, T, r, sigma, N, exercise_dates):
    """
    Calcula o preço de uma opção de compra (call) Bermudana usando o modelo de árvore binomial.

    Args:
        S0 (float): Preço inicial do ativo.
        K (float): Preço de exercício (strike).
        T (float): Tempo até o vencimento em anos.
        r (float): Taxa de juros livre de risco.
        sigma (float): Volatilidade do ativo.
        N (int): Número de passos na árvore binomial.
        exercise_dates (list): Uma lista de tempos (em anos) nos quais a opção pode ser exercida.
    """
    dt = T / N  # Tamanho de cada passo no tempo

    # Parâmetros do modelo binomial (Cox-Ross-Rubinstein)
    u = np.exp(sigma * np.sqrt(dt))  # Fator de alta
    d = 1 / u  # Fator de baixa
    p = (np.exp(r * dt) - d) / (u - d)  # Probabilidade risk-neutra de uma alta

    # Inicializa os preços do ativo no vencimento (N)
    # Cria um vetor com todos os preços possíveis do ativo no final da árvore
    asset_prices = S0 * d**np.arange(N, -1, -1) * u**np.arange(0, N + 1, 1)

    # Calcula o valor da opção no vencimento (payoff)
    option_values = np.maximum(0, asset_prices - K)

    # Converte as datas de exercício em passos de tempo correspondentes na árvore
    exercise_steps = {int(round(t / dt)) for t in exercise_dates}

    # Itera de trás para frente na árvore (do vencimento até o início)
    for i in range(N - 1, -1, -1):
        # Preços do ativo neste passo de tempo
        asset_prices = S0 * d**np.arange(i, -1, -1) * u**np.arange(0, i + 1, 1)

        # Calcula o valor de continuação (valor esperado da opção no próximo passo)
        continuation_value = np.exp(-r * dt) * (p * option_values[0:i+1] + (1 - p) * option_values[1:i+2])

        # Verifica se a data atual é uma data de exercício permitida
        if i in exercise_steps:
            # Se for, o valor da opção é o máximo entre exercer agora (valor intrínseco)
            # e esperar (valor de continuação).
            intrinsic_value = np.maximum(0, asset_prices - K)
            option_values = np.maximum(intrinsic_value, continuation_value)
        else:
            # Se não for uma data de exercício, o valor é apenas o de continuação.
            option_values = continuation_value

    # O preço da opção hoje é o primeiro valor no vetor de valores da opção
    return option_values[0]

# --- Parâmetros do Problema ---
S0 = 50.0       # Preço inicial do ativo: R$ 50
K = 50.50       # Preço de exercício: R$ 50.50
T = 3.0         # Vencimento: 3 anos
r = 0.05        # Taxa livre de risco (rf): 5%
# mu = 0.10     # O drift (μ) não é usado na precificação neutra ao risco
sigma = 0.25    # Volatilidade (σ): 25%
N = 3000        # Número de passos na árvore (para maior precisão)

# --- 1) Opção com Exercício Anual ---
# As datas de exercício são no final do ano 1, 2 e 3.
exercise_dates_yearly = [1.0, 2.0, 3.0]
price_yearly = price_bermudan_call_binomial(S0, K, T, r, sigma, N, exercise_dates_yearly)

# --- 2) Opção com Exercício Mensal ---
# Gera uma lista de datas de exercício para cada final de mês ao longo de 3 anos.
# np.arange(1/12, T + 1/12, 1/12) cria 36 pontos, de 1/12 a 3.
exercise_dates_monthly = list(np.arange(1/12, T + 1/12, 1/12))
price_monthly = price_bermudan_call_binomial(S0, K, T, r, sigma, N, exercise_dates_monthly)

# --- Apresentação dos Resultados ---
print("=" * 60)
print("Precificação de Opções de Compra (Call) Bermudanas")
print("=" * 60)
print("\nParâmetros do Modelo:")
print(f"  - Preço Inicial do Ativo (S0): R$ {S0:.2f}")
print(f"  - Preço de Exercício (K):    R$ {K:.2f}")
print(f"  - Vencimento (T):            {T} anos")
print(f"  - Taxa Livre de Risco (r):   {r:.2%}")
print(f"  - Volatilidade (σ):          {sigma:.2%}")
print("-" * 60)
print("\nResultados:")
print(f"1) Preço da Opção com Exercício Anual:  R$ {price_yearly:.4f}")
print(f"2) Preço da Opção com Exercício Mensal: R$ {price_monthly:.4f}")
print("=" * 60)


Precificação de Opções de Compra (Call) Bermudanas

Parâmetros do Modelo:
  - Preço Inicial do Ativo (S0): R$ 50.00
  - Preço de Exercício (K):    R$ 50.50
  - Vencimento (T):            3.0 anos
  - Taxa Livre de Risco (r):   5.00%
  - Volatilidade (σ):          25.00%
------------------------------------------------------------

Resultados:
1) Preço da Opção com Exercício Anual:  R$ 8.2553
2) Preço da Opção com Exercício Mensal: R$ 8.2964
